# **All-Convolutional Neural Network**

## **Libraries**

In [1]:
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import torch.nn.functional as F
import numpy as np
import torch
import time

In [2]:
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## **Settings**

In [3]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [4]:
random_seed = 1
learning_rate = 0.001
num_epochs = 15
batch_size = 256

In [5]:
num_classes = 10

## **MNIST dataset**

In [6]:
train_dataset = datasets.MNIST(root='data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='data', train=False, transform=transforms.ToTensor())

In [7]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([256, 1, 28, 28])
Image label dimensions: torch.Size([256])


## **Building the model**

In [9]:
class ConvNet(torch.nn.Module):

    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        
        self.num_classes = num_classes
        self.conv_1 = torch.nn.Conv2d(in_channels=1, out_channels=4, kernel_size=(3, 3), stride=(1, 1), padding=1) 
        self.conv_2 = torch.nn.Conv2d(in_channels=4, out_channels=4, kernel_size=(3, 3), stride=(2, 2), padding=1)                             
        self.conv_3 = torch.nn.Conv2d(in_channels=4, out_channels=8, kernel_size=(3, 3), stride=(1, 1), padding=1)                
        self.conv_4 = torch.nn.Conv2d(in_channels=8, out_channels=8, kernel_size=(3, 3), stride=(2, 2), padding=1)      
        self.conv_5 = torch.nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), stride=(1, 1), padding=1)     
        self.conv_6 = torch.nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3, 3), stride=(2, 2), padding=1)      
        self.conv_7 = torch.nn.Conv2d(in_channels=16, out_channels=self.num_classes, kernel_size=(3, 3), stride=(1, 1), padding=1)   

    def forward(self, x):
        out = self.conv_1(x)
        out = F.relu(out)
        
        out = self.conv_2(out)
        out = F.relu(out)

        out = self.conv_3(out)
        out = F.relu(out)

        out = self.conv_4(out)
        out = F.relu(out)
        
        out = self.conv_5(out)
        out = F.relu(out)
        
        out = self.conv_6(out)
        out = F.relu(out)
        
        out = self.conv_7(out)
        out = F.relu(out)
        
        logits = F.adaptive_avg_pool2d(out, 1)

        logits.squeeze_(-1)

        logits.squeeze_(-1)
        probas = torch.softmax(logits, dim=1)
        return logits, probas

In [10]:
torch.manual_seed(random_seed)
model = ConvNet(num_classes=num_classes)

In [11]:
model = model.to(device)

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

## **Training phase**

In [13]:
def compute_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0
    for features, targets in data_loader:
        features = features.to(device)
        targets = targets.to(device)
        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [14]:
start_time = time.time()
for epoch in range(num_epochs):
    model = model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        optimizer.step()
        
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))
    
    model = model.eval()
    print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
          epoch+1, num_epochs, 
          compute_accuracy(model, train_loader)))
    
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/015 | Batch 000/235 | Cost: 2.3033
Epoch: 001/015 | Batch 050/235 | Cost: 2.2850
Epoch: 001/015 | Batch 100/235 | Cost: 1.9956
Epoch: 001/015 | Batch 150/235 | Cost: 1.3309
Epoch: 001/015 | Batch 200/235 | Cost: 0.8662
Epoch: 001/015 training accuracy: 75.94%
Time elapsed: 0.74 min
Epoch: 002/015 | Batch 000/235 | Cost: 0.7992
Epoch: 002/015 | Batch 050/235 | Cost: 0.6424
Epoch: 002/015 | Batch 100/235 | Cost: 0.5292
Epoch: 002/015 | Batch 150/235 | Cost: 0.5954
Epoch: 002/015 | Batch 200/235 | Cost: 0.4468
Epoch: 002/015 training accuracy: 86.08%
Time elapsed: 1.51 min
Epoch: 003/015 | Batch 000/235 | Cost: 0.4488
Epoch: 003/015 | Batch 050/235 | Cost: 0.3600
Epoch: 003/015 | Batch 100/235 | Cost: 0.4168
Epoch: 003/015 | Batch 150/235 | Cost: 0.3020
Epoch: 003/015 | Batch 200/235 | Cost: 0.3261
Epoch: 003/015 training accuracy: 90.87%
Time elapsed: 2.25 min
Epoch: 004/015 | Batch 000/235 | Cost: 0.4119
Epoch: 004/015 | Batch 050/235 | Cost: 0.2387
Epoch: 004/015 | Batch 100

In [15]:
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Total Training Time: 10.33 min


## **Evaluation**

In [16]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 96.56%
